# Transferring Music Libraries

Minim can be used as a free, open-source alternative to services like [TuneMyMusic](https://www.tunemymusic.com/) for moving playlists and synchronizing libraries between the supported streaming services.

In [22]:
from minim import qobuz, spotify, tidal

## Prerequisites

All clients must be authenticated to access private user information. Assuming the relevant client credentials are stored as environment variables, the recommended client instantiation is as follows:

In [23]:
client_qobuz = qobuz.PrivateAPI(flow="password", browser=True)
client_spotify = spotify.WebAPI(flow="pkce", 
                                scopes=spotify.WebAPI.get_scopes("all"),
                                framework="http.server")
client_tidal = tidal.PrivateAPI(flow="device", browser=True)

## Moving playlists

The general process is to

1. get information about and the items in the source playlist,
2. create a new playlist in the destination service, and
3. find and add the corresponding items to the newly-created playlist.

The challenge often lies in the third step. The items in the source playlist may not be available in the destination service or it may be difficult finding the matching item in the destination service, especially if its catalog lookup does not support searching by ISRC.

The following examples provide barebones implementations of the process above for various service pairs. Additional fine-tuning is likely necessary to handle tracks with complex metadata, such as those with multiple (featured) artists, remixes, etc.

### From Qobuz

We start with a Qobuz playlist with 5 tracks:

In [24]:
QOBUZ_PLAYLIST_ID = 17865119

We can get the playlist information and the items in the playlist using `qobuz.PrivateAPI.get_playlist()`:

In [50]:
qobuz_playlist = client_qobuz.get_playlist(QOBUZ_PLAYLIST_ID)

#### To Spotify

First, we create a new playlist on Spotify with the same details as the Qobuz playlist using `spotify.WebAPI.create_playlist()`:

In [28]:
new_spotify_playlist = client_spotify.create_playlist(
    qobuz_playlist["name"],
    description=qobuz_playlist["description"],
    public=qobuz_playlist["is_public"],
    collaborative=qobuz_playlist["is_collaborative"],
)

Then, we get the Spotify tracks equivalent to those in the Qobuz playlist. This is a simple process as Spotify allows looking up tracks by their ISRCs with its best-in-class API:

In [54]:
spotify_track_uris = []
for qobuz_track in qobuz_playlist["tracks"]["items"]:
    spotify_track = client_spotify.search(f'isrc:{qobuz_track["isrc"]}', type="track", limit=1)["items"][0]
    spotify_track_uris.append(f"spotify:track:{spotify_track['id']}")

Finally, we add the tracks to the Spotify playlist using `spotify.WebAPI.add_playlist_items()`:

In [55]:
client_spotify.add_playlist_items(new_spotify_playlist["id"], spotify_track_uris)

In [ ]:
client_spotify.unfollow_playlist(new_spotify_playlist["id"])

#### To TIDAL

First, we create a new playlist on TIDAL with the same details as the Qobuz playlist using `tidal.PrivateAPI.create_playlist()`:

In [ ]:
new_tidal_playlist = client_tidal.create_playlist(
    qobuz_playlist["name"], 
    description=qobuz_playlist["description"],
    public=qobuz_playlist["is_public"]
)

Then, we try to find TIDAL tracks equivalent to those in the Qobuz playlist. Unfortunately, TIDAL does not support searching by ISRCs, so we have to look up the tracks using their titles and artists. The TIDAL API does, however, return ISRCs so we can confirm that we have the right tracks before adding them to the TIDAL playlist.

In [67]:
tidal_track_ids = []
for qobuz_track in qobuz_playlist["tracks"]["items"]:
    title = qobuz_track["title"]
    if qobuz_track["version"]:
        title += f' {qobuz_track["version"]}'
    tidal_track = client_tidal.search(
        f'{qobuz_track["performer"]["name"]} {title}', 
        type="track", 
        limit=1
    )["items"][0]
    if qobuz_track["isrc"] == tidal_track["isrc"]:
        tidal_track_ids.append(tidal_track["id"])

Finally, we add the tracks to the TIDAL playlist using `tidal.PrivateAPI.add_playlist_items()`:

In [ ]:
client_tidal.add_playlist_items(new_tidal_playlist["data"]["uuid"], tidal_track_ids)

In [ ]:
client_tidal.delete_playlist(new_tidal_playlist["data"]["uuid"])

### From Spotify

We start with a Spotify playlist with 5 tracks:

In [ ]:
SPOTIFY_PLAYLIST_ID = "3rw9qY60CEh6dfJauWdxMh"

We can get the playlist information and the items in the playlist using `spotify.WebAPI.get_playlist()`:

In [ ]:
spotify_playlist = client_spotify.get_playlist(SPOTIFY_PLAYLIST_ID)

#### To Qobuz

First, we create a new playlist on Qobuz with the same details as the Spotify playlist using `qobuz.PrivateAPI.create_playlist()`:

In [ ]:
new_qobuz_playlist = client_qobuz.create_playlist(
    spotify_playlist["name"],
    description=spotify_playlist["description"],
    public=spotify_playlist["public"],
    collaborative=spotify_playlist["collaborative"],
)

Then, we get the Qobuz tracks equivalent to those in the Spotify playlist. Thankfully, we can search by ISRC on Qobuz, so we can get the correct Qobuz tracks directly if they are available in the Qobuz catalog:

In [ ]:
qobuz_track_ids = []
for spotify_track in spotify_playlist["tracks"]["items"]:
    qobuz_track = client_qobuz.search(
        spotify_track["track"]["external_ids"]["isrc"],  
        limit=1
    )["tracks"]["items"][0]
    qobuz_track_ids.append(qobuz_track["id"])

Finally, we add the tracks to the Qobuz playlist using `qobuz.PrivateAPI.add_playlist_tracks()`:

In [ ]:
client_qobuz.add_playlist_tracks(new_qobuz_playlist["id"], qobuz_track_ids)

In [ ]:
client_qobuz.delete_playlist(new_qobuz_playlist["id"])

#### To TIDAL

First, we create a new playlist on TIDAL with the same details as the Spotify playlist:

In [ ]:
new_tidal_playlist = client_tidal.create_playlist(
    spotify_playlist["name"], 
    description=spotify_playlist["description"],
    public=spotify_playlist["public"]
)

Then, we try to find TIDAL tracks equivalent to those in the Spotify playlist:

In [ ]:
tidal_track_ids = []
for spotify_track in spotify_playlist["tracks"]["items"]:
    tidal_track = client_tidal.search(
        f'{spotify_track["track"]["artists"][0]["name"]} '
        f'{spotify_track["track"]["name"]}', 
        type="track", 
        limit=1
    )["items"][0]
    if spotify_track["track"]["external_ids"]["isrc"] == tidal_track["isrc"]:
        tidal_track_ids.append(tidal_track["id"])

Finally, we add the tracks to the TIDAL playlist:

In [ ]:
client_tidal.add_playlist_items(new_tidal_playlist["data"]["uuid"], tidal_track_ids)

In [ ]:
client_tidal.delete_playlist(new_tidal_playlist["data"]["uuid"])

### From TIDAL

#### To Qobuz

#### To Spotify

## Synchronizing favorites